In [79]:
import random
import subprocess
import json
import numpy as np
from tqdm import tqdm
import time
import os

In [80]:
# Print win records between matches
PRINT_PROGRESS = False

In [81]:
# read in ai bot file names
ais = os.listdir('./models')
ais = [ai for ai in ais if ai.endswith('.h5')]
# how many ai models
num_versions = len(ais)

In [82]:
bots = list(filter(lambda b:b.endswith('.py'), os.listdir('../bots')))
for file in bots:
    if file.endswith('.py'):
        os.rename('../bots/'+file, '../bots/'+file.replace(' ',''))

if len(ais)>len(list(filter(lambda b:b.startswith('MyBot') and b.endswith(').py'), bots))):
    raise Exception('Check number of ai\'s vs MyBot files')

In [83]:
# list of enemy bot files to choose from
bot_list = []

In [84]:
bot_list.extend([
    "mining","mining","mining",
    "mine_attack", "mine_attack","mine_attack","mine_attack",
    "aggressive", "aggressive","aggressive","aggressive",
    "mine_attack_later", "mine_attack_later","mine_attack_later","mine_attack_later",
    ])

In [85]:
bot_list.extend([
    "random0","random0",
    "random1","random1",
    "random3","random3",
    "random5","random5",
    "random10","random10",
    "random15","random15",
    "random20","random20",
    ])

In [86]:
# track wins per ai model
wins = dict({i:0 for i in list(range(num_versions))+list(set(bot_list))})
# track matches per ai model
num = dict({i:0 for i in list(range(num_versions))+list(set(bot_list))})

In [87]:
def print_progress():
    out = ''
    for idx, i in enumerate(ai_versions+list(set(bot_list))):
        if np.mod(idx,4)==0:
            out += '\n'
        try:
            out += '{}:{}/{}={}%\t'.format(i,wins[i],num[i],round(100*wins[i]/num[i],1))
        except ZeroDivisionError:
            out += '{}:{}/0=0%\t'.format(i,wins[i])
    tqdm.write(out)

In [88]:
def cleanup():
    files = os.listdir('.')
    for file in files:
        if file.endswith('.vec') or file.endswith('.log'):
            os.remove(file)

In [ ]:
# run number of matches or until stop.txt exists (whichever happens first)
for m in tqdm(range(100)):
    g2g = False
    
    while not g2g:
        # build player list for the match
        cmd = 'halite.exe'
        # store enemy type ('na' or ai version)
        enemy_version = []
        # list of ai versions to choose from
        ai_versions = list(range(num_versions))
        
        # add 2 or 4 players
        for _ in range(random.choice([2, 4])):
            # randomly choose from from ai or botlist
            enemy_version.append(random.choice(ai_versions+['na']*len(bot_list)))
            
            # build player into cmd string
            if enemy_version[-1] == 'na':
                # bot_list player
                player = random.choice(bot_list)
                cmd+=' "python ../bots/{}.py"'.format(player)
                enemy_version[-1] = player
                num[enemy_version[-1]] += 1
            else:
                # ai player
                cmd+=' "python ../bots/MyBot-Copy({}).py"'.format(enemy_version[-1])
                num[enemy_version[-1]] += 1
                g2g = True
        
    cmd+=' -q -t --constantsfile "constants.txt"'
    
    # run match
    result = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT).stdout.read()
    # translate results from json
    try:
        result = json.loads(result)
    except:
        print(result)
        break
    
    # ignore matches where there's an error
    if not len(result['error_logs']):
        # sort players by rank
        ranked = sorted(result['stats'].items(), key=lambda t:t[1]['rank'])
        
        # get best ranked player
        for player in ranked:
            winner_id = int(player[0])
            winner = enemy_version[winner_id]
            break
        
        ## filter winners
        #if (isinstance(winner, str) and winner.startswith('random')) or isinstance(winner, int):
        # get inputs
        with open("{}_{}_input.vec".format(winner_id, winner), "r") as f:
            input_lines = f.readlines()

        # get outputs
        with open("{}_{}_out.vec".format(winner_id, winner), "r") as f:
            output_lines = f.readlines()

        # check lengths, append to training data
        if len(input_lines)==len(output_lines):
            with open("train.in", "a") as f:
                for l in input_lines:
                    f.write(l)
            with open("train.out", "a") as f:
                for l in output_lines:
                    f.write(l)
        else:
            print(result)
            print(enemy_version)
            print_progress()
            raise('Bot data lengths do not match')
        ## filter winners
        #else:
        os.remove(result['replay'])
        
        # track wins for winner
        wins[winner] += 1
        
        # print as we go
        if PRINT_PROGRESS:
            tqdm.write('{} id {} won in {}'.format(winner, winner_id, enemy_version))
            tqdm.write(result['replay'])
            tqdm.write("\nWins")
            print_progress()
        
        cleanup()
        
        if os.path.isfile('stop.txt'):
            break
    
    else:
        #cleanup()
        
        print("\nWins")
        print_progress()
        
        print(result)
        print(enemy_version)
        print(result['replay'])
        break
    
    time.sleep(1)
    
print('\n\n****** FINAL SCORES ******')
print_progress()


 64%|██████████████████████████████████████████████████▌                            | 64/100 [1:16:10<39:47, 66.33s/it]

In [67]:
cmd

'halite.exe "python ../bots/random20.py" "python ../bots/MyBot-Copy(17).py" "python ../bots/MyBot-Copy(14).py" "python ../bots/MyBot-Copy(1).py" -q --constantsfile "constants.txt"'